## Null Value treatment for Train_Claim Using Model Based Imputation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import joblib

import warnings
warnings.filterwarnings("ignore")

In [2]:
dfc = pd.read_csv(r"C:\Users\Soorya\Documents\capstone-files\TrainData\Data\Train_Claim.csv",na_values= [ "?", "-5", "MISSINGVALUE", "MISSEDDATA"])
print(dfc.shape)
dfc.head()

(28836, 19)


,CustomerID,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,PropertyDamage,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage
0,Cust10000,2015-02-03,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City1,Location 1311,17.0,3,NaN,1,0.0,NaN,65501.0,13417,6071,46013
1,Cust10001,2015-02-02,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City5,Location 1311,10.0,3,YES,2,1.0,YES,61382.0,15560,5919,39903
2,Cust10002,2015-01-15,Single Vehicle Collision,Side Collision,Minor Damage,Other,State8,City6,Location 2081,22.0,1,YES,2,3.0,NO,66755.0,11630,11630,43495
3,Cust10003,2015-01-19,Single Vehicle Collision,Side Collision,Minor Damage,Other,State9,City6,Location 2081,22.0,1,YES,2,3.0,NO,66243.0,12003,12003,42237
4,Cust10004,2015-01-09,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State8,City6,Location 1695,10.0,1,NO,2,1.0,YES,53544.0,8829,7234,37481


In [3]:
dfc.isnull().sum()

CustomerID                   0
DateOfIncident               0
TypeOfIncident               0
TypeOfCollission          5162
SeverityOfIncident           0
AuthoritiesContacted         0
IncidentState                0
IncidentCity                 0
IncidentAddress              0
IncidentTime                31
NumberOfVehicles             0
PropertyDamage           10459
BodilyInjuries               0
Witnesses                   46
PoliceReport              9805
AmountOfTotalClaim          50
AmountOfInjuryClaim          0
AmountOfPropertyClaim        0
AmountOfVehicleDamage        0
dtype: int64

In [4]:
# -1 for No witness info in the dataset
dfc['Witnesses'].fillna(-1,axis=0,inplace=True)

In [5]:
# median imputation for the AmountOfTotalClaim
dfc['AmountOfTotalClaim'].fillna(dfc['AmountOfTotalClaim'].median(),axis=0,inplace=True)

In [6]:
# null value is replaced with "NoReport"
dfc['PoliceReport'].fillna("NoReport",axis=0,inplace=True)
dfc['PoliceReport'].value_counts()

NO          9898
NoReport    9805
YES         9133
Name: PoliceReport, dtype: int64

In [7]:
# null value is replaced with "Missing collision"
dfc['TypeOfCollission'].fillna("Missing Info",axis=0,inplace=True)

dfc['TypeOfCollission'].value_counts()

Rear Collision     8561
Side Collision     7867
Front Collision    7246
Missing Info       5162
Name: TypeOfCollission, dtype: int64

In [8]:
# null value is replaced with -10.0
dfc['IncidentTime'].fillna(-10.0,axis=0,inplace=True)

In [9]:
dfc["Year"] = dfc['DateOfIncident'].apply(lambda x: int(x.split("-")[0]))
dfc["Month"] = dfc['DateOfIncident'].apply(lambda x: int(x.split("-")[1]))
dfc["days"] = dfc['DateOfIncident'].apply(lambda x: int(x.split("-")[2]))

dfc.drop("DateOfIncident",axis=1,inplace=True)

In [10]:
dfc.isnull().sum().sum()

10459

## Model Based Null value Imputation for PropertyDamage column

In [11]:
#Before building a model lets perform data pre-processing

# seperating the rows with not-null values
df2 = dfc.dropna().copy()

In [12]:
print(df2.shape)
df2.head()

(18377, 21)


,CustomerID,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,...,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,Year,Month,days
1,Cust10001,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City5,Location 1311,10.0,3,...,2,1.0,YES,61382.0,15560,5919,39903,2015,2,2
2,Cust10002,Single Vehicle Collision,Side Collision,Minor Damage,Other,State8,City6,Location 2081,22.0,1,...,2,3.0,NO,66755.0,11630,11630,43495,2015,1,15
3,Cust10003,Single Vehicle Collision,Side Collision,Minor Damage,Other,State9,City6,Location 2081,22.0,1,...,2,3.0,NO,66243.0,12003,12003,42237,2015,1,19
4,Cust10004,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State8,City6,Location 1695,10.0,1,...,2,1.0,YES,53544.0,8829,7234,37481,2015,1,9
5,Cust10005,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State7,City6,Location 1695,7.0,1,...,1,2.0,NoReport,53167.0,7818,8132,37217,2015,2,7


In [13]:
# index value for filtering rows with null values
df2_index = set(df2.index.values)
dfc_index = set(dfc.index.values)
df3_index = dfc_index.difference(df2_index)

In [14]:
df3 = dfc.filter(items=list(df3_index), axis=0).copy()

In [15]:
# all rows with null values
df3.head()

,CustomerID,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,...,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,Year,Month,days
0,Cust10000,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City1,Location 1311,17.0,3,...,1,0.0,NoReport,65501.0,13417,6071,46013,2015,2,3
6,Cust10006,Single Vehicle Collision,Front Collision,Minor Damage,Ambulance,State5,City4,Location 1440,20.0,1,...,0,2.0,NO,77453.0,6476,12822,58155,2015,1,30
7,Cust10007,Single Vehicle Collision,Front Collision,Minor Damage,Police,State5,City3,Location 1440,18.0,1,...,0,2.0,NO,60569.0,5738,7333,47498,2015,1,12
15,Cust10017,Multi-vehicle Collision,Side Collision,Total Loss,Other,State7,City2,Location 1677,12.0,1,...,1,0.0,YES,61697.0,6172,6167,49358,2015,2,28
18,Cust10021,Vehicle Theft,Missing Info,Trivial Damage,None,State4,City5,Location 1449,10.0,1,...,2,1.0,NoReport,6190.0,619,1208,4363,2015,1,3


In [16]:
df3.shape

(10459, 21)

In [17]:
dfc_notnull = df2.copy()
dfc_null = df3.copy()

## Perform data preparation 

In [18]:
df2['PropertyDamage'].head()

1    YES
2    YES
3    YES
4     NO
5     NO
Name: PropertyDamage, dtype: object

In [19]:
le = LabelEncoder()
df2['CustomerID'] = le.fit_transform(df2['CustomerID'])
df2['IncidentAddress'] = le.fit_transform(df2['IncidentAddress'])

df3['CustomerID'] = le.fit_transform(df3['CustomerID'])
df3['IncidentAddress'] = le.fit_transform(df3['IncidentAddress'])

In [20]:
for i in  df2.columns:
    if df2[i].dtype == 'object':
        df2[i] = df2[i].astype("category").cat.codes
        
df3.drop(["PropertyDamage"],axis=1,inplace=True)
for i in  df3.columns:
    if df3[i].dtype == 'object':
        df3[i] = df3[i].astype("category").cat.codes

In [21]:
x = df2.drop('PropertyDamage',axis=1)
y = df2['PropertyDamage']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [22]:
k = list(range(1,30,2))

knn_acc = []

for i in k:
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(x_train,y_train)
    knn_acc.append(neigh.score(x_test, y_test))

In [23]:
print("The optimal value for k is",k[np.argmax(np.array(knn_acc))],"with accuracy score",np.max(np.array(knn_acc)))

The optimal value for k is 1 with accuracy score 0.6183351468988031


In [24]:
k = [10**x for x in range(-4,5)]

lr_acc = []

for i in k:
    lr = LogisticRegression(C=i)
    lr.fit(x_train,y_train)
    lr_acc.append(lr.score(x_test, y_test))

In [25]:
print("The optimal value for k is",k[np.argmax(np.array(lr_acc))],"with accuracy score",np.max(np.array(lr_acc)))

The optimal value for k is 0.1 with accuracy score 0.5389009793253536


In [26]:
# Training KNN with optimal K=1 on entire dataset 
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(x,y)

KNeighborsClassifier(n_neighbors=1)

In [27]:
joblib.dump(neigh, 'policyClaim.joblib')

['policyClaim.joblib']

## predict for null values 

In [28]:
pred = neigh.predict(df3)
pred

array([1, 1, 0, ..., 1, 1, 1], dtype=int8)

In [29]:
#adding predictions to dfd_null dataframe
dfc_null['PropertyDamage'] = pred

In [30]:
dfc_null['PropertyDamage'] = dfc_null.PropertyDamage.apply(lambda x: "YES" if x else "NO")

In [31]:
dfc_null.head()

,CustomerID,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,...,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,Year,Month,days
0,Cust10000,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City1,Location 1311,17.0,3,...,1,0.0,NoReport,65501.0,13417,6071,46013,2015,2,3
6,Cust10006,Single Vehicle Collision,Front Collision,Minor Damage,Ambulance,State5,City4,Location 1440,20.0,1,...,0,2.0,NO,77453.0,6476,12822,58155,2015,1,30
7,Cust10007,Single Vehicle Collision,Front Collision,Minor Damage,Police,State5,City3,Location 1440,18.0,1,...,0,2.0,NO,60569.0,5738,7333,47498,2015,1,12
15,Cust10017,Multi-vehicle Collision,Side Collision,Total Loss,Other,State7,City2,Location 1677,12.0,1,...,1,0.0,YES,61697.0,6172,6167,49358,2015,2,28
18,Cust10021,Vehicle Theft,Missing Info,Trivial Damage,None,State4,City5,Location 1449,10.0,1,...,2,1.0,NoReport,6190.0,619,1208,4363,2015,1,3


In [32]:
dfc_null['PropertyDamage'].value_counts()

NO     5751
YES    4708
Name: PropertyDamage, dtype: int64

In [33]:
# appending the 2 dataframe to get the imputed null values for demographics data
dfc_final = dfc_notnull.append(dfc_null)

dfc_final.shape

(28836, 21)

In [34]:
dfc_final.head()

,CustomerID,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,...,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,Year,Month,days
1,Cust10001,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City5,Location 1311,10.0,3,...,2,1.0,YES,61382.0,15560,5919,39903,2015,2,2
2,Cust10002,Single Vehicle Collision,Side Collision,Minor Damage,Other,State8,City6,Location 2081,22.0,1,...,2,3.0,NO,66755.0,11630,11630,43495,2015,1,15
3,Cust10003,Single Vehicle Collision,Side Collision,Minor Damage,Other,State9,City6,Location 2081,22.0,1,...,2,3.0,NO,66243.0,12003,12003,42237,2015,1,19
4,Cust10004,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State8,City6,Location 1695,10.0,1,...,2,1.0,YES,53544.0,8829,7234,37481,2015,1,9
5,Cust10005,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State7,City6,Location 1695,7.0,1,...,1,2.0,NoReport,53167.0,7818,8132,37217,2015,2,7


In [35]:
dfc_final.to_csv("PolicyClaim.csv",index=False)